##### IMPORTS


In [28]:
import pandas as pd
import wfdb
import numpy as np
import matplotlib.pyplot as plt 
# from statsmodels.tsa.arima_model import ARIMA
# from statsmodels.tsa.arima_model import ARIMAResults
import statsmodels.api as sm
from darts import TimeSeries
from darts.metrics.metrics import mae as meanabs ,mse,mape, r2_score
from darts.models import NBEATSModel
import warnings
warnings.filterwarnings('ignore')
from sktime.forecasting.compose import make_reduction, TransformedTargetForecaster
from sktime.forecasting.model_selection import ExpandingWindowSplitter, ForecastingGridSearchCV
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
import lightgbm as lgb
import sklearn
from sklearn.linear_model import Lasso,LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error, mean_squared_log_error
import xgboost
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
tf.random.set_seed(7)
import torch

In [12]:
ecg = []
# for i in range(10,13):
#     for j in range(1,20):
#         if i<10:
#             path = "../dataset/ECGData/Person_0{0}/rec_{1}".format(i,j)
#         else: 
#             path = "../dataset/ECGData/Person_{0}/rec_{1}".format(i,j)
#         try:    
#             record = wfdb.rdsamp(path)
#             # data = np.array(record[0])
#             data = record[0]
#             data = data[:,1]
            
#             ecg.append(data)
#             c+=1
#         except:
#             continue

for j in range(100,235):
# for j in range(100,105):
    path = "../dataset/MIT-BIH/{}".format(j)
    try:    
        record = wfdb.rdsamp(path)
        # data = np.array(record[0])
        data = record[0]
        data = data[:,1]
        # scaler = StandardScaler()
        # ecg.append(scaler.fit_transform(data.reshape(-1,1)))
        ecg.append(data)
    
    except:
        continue

# for j in range(10,12):
#     if j<10:
#         path = "../dataset/FetalECG/ARR_0{}".format(j)
#     else:
#         path = "../dataset/FetalECG/ARR_{}".format(j)
#     try:    
#         record = wfdb.rdsamp(path)
#         # data = np.array(record[0])
#         data = record[0]
#         data = data[:,0]
        
#         ecg.append(data)
        
#     except:
#         continue

# for j in range(12,14):
#     if j<10:
#         path = "../dataset/FetalECG/NR_0{}".format(j)
#     else:
#         path = "../dataset/FetalECG/NR_{}".format(j)
#     try:    
#         record = wfdb.rdsamp(path)
#         # data = np.array(record[0])
#         data = record[0]
#         data = data[:,0]
        
#         ecg.append(data)
       
#     except:
#         continue

# for j in range(1,2):
#     path = "../dataset/PretermECG/infant{}_ecg".format(j)
#     try:    
#         record = wfdb.rdsamp(path)
#         # data = np.array(record[0])
#         data = record[0]
#         data = data[:,0]
        
#         ecg.append(data)
        
        
#     except:
#         continue

ecg = np.array(ecg)
Data_ =[]
for i in range(len(ecg)):
    temp = [[t,ecg[i][t]] for t in range(len(ecg[i]))]
    Data_.append(temp)


In [13]:
def create_dataset(dataset, look_back=50):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

### ARIMA

In [14]:
rmsle =[]
rmse =[]
r2 =[]
mae = []

for i in Data_:
    i= np.array(i)
    X = i[:,1]
    # scaler = StandardScaler()
    X = X.reshape(-1,1)
    split_len = int(0.8*len(X))
    X_train,y_test = X[:split_len] , X[split_len:]
    model = sm.tsa.arima.ARIMA(X_train, order=(4,1,0))
    model_fit = model.fit()
    pred = model_fit.forecast(len(y_test))
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print( 'rmsle: ',np.mean(rmsle))
# model = sm.tsa.arima.ARIMA(ecg[29][:-2000], order=(4,1,0))
# model_fit = model.fit()
# plt.plot(ecg[29][-2000:],'r')
# # model_fit.predict()
# plt.plot(model_fit.forecast(2000),'g')
# plt.show()
# print(sum(abs(ecg[29][-2000:]-model_fit.forecast(2000)))**2/len(ecg[23]))


mae:  0.24863613180828933
rmse:  0.18484619731012394
r2:  -0.5303065015740012


In [15]:
rmsle =[]
rmse =[]
r2 =[]
mae = []

for i in Data_:
    i= np.array(i)
    X = i[:,1]
    scaler = StandardScaler()
    X = scaler.fit_transform(X.reshape(-1,1))
    split_len = int(0.8*len(X))
    X_train,y_test = X[:split_len] , X[split_len:]
    model = sm.tsa.arima.ARIMA(X_train, order=(4,1,0))
    model_fit = model.fit()
    pred = model_fit.forecast(len(y_test))
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print( 'rmsle: ',np.mean(rmsle))
# model = sm.tsa.arima.ARIMA(ecg[29][:-2000], order=(4,1,0))
# model_fit = model.fit()
# plt.plot(ecg[29][-2000:],'r')
# # model_fit.predict()
# plt.plot(model_fit.forecast(2000),'g')
# plt.show()
# print(sum(abs(ecg[29][-2000:]-model_fit.forecast(2000)))**2/len(ecg[23]))


mae:  0.8285750099788967
rmse:  1.543666916550448
r2:  -0.5303065029224769


#### LightGBM

In [16]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = lgb.LGBMRegressor()
    # scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = i.reshape(-1,1)
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    # plt.plot(pred[0:1000],'r')
    # plt.plot(y_test[0:1000],'g')
    # break
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.016333867884851628
rmse:  0.008103178821084968
r2:  0.9643295970811812


In [17]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = lgb.LGBMRegressor()
    scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = scaler.transform(i.reshape(-1,1))
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    # plt.plot(pred[0:1000],'r')
    # plt.plot(y_test[0:1000],'g')
    # break
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.05984980579300179
rmse:  0.042214161001337125
r2:  0.9654806408729478


### LASSO

In [18]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i= np.array(i)
    lasso = Lasso(0.01)
    # scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = i.reshape(-1,1)
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.04307645944260089
rmse:  0.006701546596643289
r2:  0.79364205945286


In [19]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i= np.array(i)
    lasso = Lasso(0.01)
    scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = scaler.transform(i.reshape(-1,1))
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.07307049605210002
rmse:  0.015806188154845528
r2:  0.9840135783599019


### LINEAR REGRESSION

In [20]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = LinearRegression()
    # scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = i.reshape(-1,1)
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.01302615240967746
rmse:  0.00036370334429897464
r2:  0.9922780153127139


In [21]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = LinearRegression()
    scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = scaler.transform(i.reshape(-1,1))
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    #rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
#print('rmsle: ',np.mean(rmsle))

mae:  0.05627654837485021
rmse:  0.0073897016321069235
r2:  0.9922780153127135


### XGBoost

In [22]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = xgboost.XGBRegressor()
    # scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = i.reshape(-1,1)
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    print(i)
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print('rmsle: ',np.mean(rmsle))

[-0.065 -0.065 -0.065 ... -0.365 -0.335  0.   ]
[-0.16 -0.16 -0.16 ... -0.11 -0.11  0.  ]
[0.005 0.005 0.005 ... 0.2   0.195 0.   ]
[0.05  0.05  0.05  ... 0.05  0.055 0.   ]
[0.2   0.2   0.2   ... 0.2   0.205 0.   ]
[ 0.26  0.26  0.26 ... -0.37 -0.38  0.  ]
[0.195 0.195 0.195 ... 0.265 0.405 0.   ]
[-1.245 -1.245 -1.245 ...  0.07   0.065  0.   ]
[-0.78  -0.78  -0.78  ...  0.085  0.08   0.   ]
[0.635 0.635 0.635 ... 0.36  0.365 0.   ]
[ 0.035  0.035  0.035 ... -0.79  -0.795 -1.28 ]
[-0.62 -0.62 -0.62 ... -0.59 -0.58  0.  ]
[0.13 0.13 0.13 ... 0.03 0.03 0.  ]
[ 0.015  0.015  0.015 ... -0.215 -0.225  0.   ]
[-0.335 -0.335 -0.335 ...  0.24   0.24   0.   ]
[-1.24  -1.24  -1.24  ... -0.715 -0.715 -1.28 ]
[-0.47  -0.47  -0.47  ... -0.225 -0.245  0.   ]
[-0.515 -0.515 -0.515 ... -2.1   -2.14  -2.56 ]
[-0.47  -0.47  -0.47  ... -1.825 -1.74  -1.28 ]
[-0.65  -0.65  -0.65  ... -0.695 -0.665 -1.28 ]
[-0.735 -0.735 -0.735 ... -0.58  -0.585  0.   ]
[-0.455 -0.455 -0.455 ... -0.705 -0.69  -1.28 ]
[-0.

In [23]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = xgboost.XGBRegressor()
    scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = scaler.transform(i.reshape(-1,1))
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print('rmsle: ',np.mean(rmsle))

mae:  0.05699804225654694
rmse:  0.03831002817367366
r2:  0.9690764025657513


### Bagging Regression

In [24]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = BaggingRegressor()
    # scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = i.reshape(-1,1)
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(np.sqrt(mean_squared_error(y_test,pred)))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print('rmsle: ',np.mean(rmsle))

mae:  0.015526374432526418
rmse:  0.037534631709022534
r2:  0.969301940338334


In [25]:
rmsle =[]
rmse =[]
r2 =[]
mae = []
for i in ecg:
    i = np.array(i)
    lasso = BaggingRegressor()
    scaler = StandardScaler().fit(i.reshape(-1,1))
    temp_data = scaler.transform(i.reshape(-1,1))
    # X = i[:,0]
    # Y = i[:,1] 
    train_size = int(len(temp_data) * 0.3)
    test_size = len(temp_data) - train_size
    train, test = temp_data[0:train_size,:], temp_data[train_size:len(temp_data),:]

    ### train aur test column honge shayad
    X_train, y_train = create_dataset(train)
    X_test, y_test = create_dataset(test)
    # X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,  shuffle=False)

    lasso = lasso.fit(X_train,y_train)
    pred = lasso.predict(X_test)
    # rmsle.append(mean_squared_log_error(y_test,pred))
    rmse.append(np.sqrt(mean_squared_error(y_test,pred)))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    
print("mae: ",np.mean(mae))
print('rmse: ',np.mean(rmse))
print('r2: ',np.mean(r2))
# print('rmsle: ',np.mean(rmsle))

mae:  0.058200658611450164
rmse:  0.12447187352322335
r2:  0.969389934572724


### N Beats

In [26]:

Data =[]
for i in range(len(ecg)):
    scaler = StandardScaler().fit_transform(ecg[i].reshape(-1,1))
    temp = [[t,scaler[t]] for t in range(len(ecg[i]))]
    temp = TimeSeries.from_values(np.array(temp))
    print(i)
    Data.append(temp)
np.random.shuffle(Data)
train = Data[:int(len(Data)*0.8)]
test =  Data[int(len(Data)*0.8):]
    

torch.set_float32_matmul_precision('medium')
Nbeats = NBEATSModel(
    input_chunk_length=30,
    output_chunk_length=7,
    generic_architecture=True,
    num_stacks=8,
    num_blocks=1,
    num_layers=3,
    layer_widths=256,
    n_epochs=10,
    nr_epochs_val_period=1,
    batch_size=500,
    model_name="nbeats",
    pl_trainer_kwargs ={"accelerator": "gpu", "devices": -1}
)

Nbeats = Nbeats.fit(train)

Nbeats.save('../models/Nbeats_normalized.pt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 1.2 M 
---------------------------------------------------
1.2 M     Trainable params
1.6 K     Non-trainable params
1.2 M     Total params
4.807     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 49398/49398 [33:42<00:00, 24.43it/s, train_loss=1.63e+3]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 49398/49398 [33:42<00:00, 24.43it/s, train_loss=1.63e+3]


In [30]:
torch.set_float32_matmul_precision('medium')
Nbeats = NBEATSModel.load('../models/Nbeats_normalized.pt')
rmse_ = []
mae_ = []
r2_ = []
i = int((1/2)*len(Data[1]))

for i in test:
    n = int(len(i)*0.2)
    tot = len(i)-n
    pred = Nbeats.predict(n,i[:tot])
    y = i[tot:]
    rmse_.append(np.sqrt(mse(y,pred)))
    mae_.append(meanabs(y,pred))
    r2_.append(r2_score(y,pred))
# print(sum((y-pred)**2)/len(y))

print('rmse: ',np.mean(rmse_))
print('mae: ',np.mean(mae_))
print('mae: ',np.mean(r2_))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 1/1 [01:15<00:00, 75.61s/it]


In [ ]:

Data =[]
for i in range(len(ecg)):
    scaler = ecg[i].reshape(-1,1)
    temp = [[t,scaler[t]] for t in range(len(ecg[i]))]
    temp = TimeSeries.from_values(np.array(temp))
    print(i)
    Data.append(temp)
np.random.shuffle(Data)
train = Data[:int(len(Data)*0.8)]
test =  Data[int(len(Data)*0.8):]
    

torch.set_float32_matmul_precision('medium')
Nbeats = NBEATSModel(
    input_chunk_length=30,
    output_chunk_length=7,
    generic_architecture=True,
    num_stacks=8,
    num_blocks=1,
    num_layers=3,
    layer_widths=256,
    n_epochs=10,
    nr_epochs_val_period=1,
    batch_size=500,
    model_name="nbeats",
    pl_trainer_kwargs ={"accelerator": "gpu", "devices": -1}
)

Nbeats = Nbeats.fit(train)

Nbeats.save('../models/Nbeats_.pt')


In [ ]:
torch.set_float32_matmul_precision('medium')
Nbeats = NBEATSModel.load('../models/Nbeats.pt')
rmse_ = []
mae_ = []
r2_ = []
i = int((1/2)*len(Data[1]))

for i in test:
    n = int(len(i)*0.2)
    tot = len(i)-n
    pred = Nbeats.predict(n,i[:tot])
    y = i[tot:]
    rmse_.append(np.sqrt(mse(y,pred)))
    mae_.append(meanabs(y,pred))
    r2_.append(r2_score(y,pred))
# print(sum((y-pred)**2)/len(y))

print('rmse: ',np.mean(rmse_))
print('mae: ',np.mean(mae_))
print('mae: ',np.mean(r2_))


### SARIMAX


In [ ]:

rmsle =[]
mse =[]
r2 =[]
mae = []

for i in Data_:
    i= np.array(i)
    X = i[:,1]
    # scaler = StandardScaler()
    X = X.reshape(-1,1)
    split_len = int(0.8*len(X))
    X_train,y_test = X[:split_len] , X[split_len:]
    model = sm.tsa.statespace.SARIMAX(X_train, order=(7,1,7))
    
    model_fit = model.fit()
    
    pred = model_fit.forecast(len(y_test))
   
    #rmsle.append(mean_squared_log_error(y_test,pred))
    mse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    

print("mae: ",np.mean(mae))
print('mse: ',np.mean(mse))
print('r2: ',np.mean(r2))
#print( 'rmsle: ',np.mean(rmsle))

In [ ]:

rmsle =[]
mse =[]
r2 =[]
mae = []

for i in Data_:
    i= np.array(i)
    X = i[:,1]
    scaler = StandardScaler()
    X = scaler.fit_transform(X.reshape(-1,1))
    split_len = int(0.8*len(X))
    X_train,y_test = X[:split_len] , X[split_len:]
    model = sm.tsa.statespace.SARIMAX(X_train, order=(7,1,7))
    
    model_fit = model.fit()
    
    pred = model_fit.forecast(len(y_test))
   
    #rmsle.append(mean_squared_log_error(y_test,pred))
    mse.append(mean_squared_error(y_test,pred))
    r2.append(r2_score(y_test,pred))
    mae.append(mean_absolute_error(y_test,pred))
    

print("mae: ",np.mean(mae))
print('mse: ',np.mean(mse))
print('r2: ',np.mean(r2))
#print( 'rmsle: ',np.mean(rmsle))